In [5]:
import torch
from torch.nn import functional as F

In [3]:
def frozen_kmeans(self, cached, fronzen_state, kmeans_iters=5):
    frozen_length = torch.count_non_zero(fronzen_state)
    unfrozen_length = fronzen_state.shape[0] - frozen_length
    unique_vecs = cached.unique(dim=0)

    random_idx = torch.randperm(unique_vecs.size(dim=0))[:unfrozen_length]
    self.embedding[torch.logical_not(fronzen_state)] = cached[random_idx]

    ## Initialisaation complete move on to algorithm

    for _ in range(kmeans_iters):
        distances = torch.sum(cached**2, dim=1, keepdim=True) + torch.sum(self.embedding, dim=1) - 2.0 * torch.matmul(cached, self.embedding.t())  # calculate the distances from each centroid to the value
        dists, indexes = torch.min(distances, dim=1)

        counts = torch.bincount(indexes)  # count the number in each bin
        counts[fronzen_state] = -1  # make sure to mark as frozen

        ### First deal with those with no nearby points ###
        #  get the indices of the points furthest away from their centroid
        indices = torch.sort(dists).indices[:torch.sum(counts==0)]  # torch.sum(Tensor[bool]) is short hand for count number of trues
        self.embedding[counts == 0] = self.cache[indices]


In [24]:

original =  torch.Tensor([5,6,7]).to(torch.int64)
indices = torch.Tensor([1,1,1]).to(torch.int64)
### Then deal with the means ###
oh = F.one_hot(indices, num_classes=3).t() 
amount_of_each =  torch.sum(oh, dim=1)

torch.matmul(oh, original)

tensor([ 0, 18,  0])